In [2]:
# Record live audio 
from pyaudio import PyAudio, paInt16
import wave
import os
import time
import sys
import soundfile as sf
import sounddevice as sd



# Record live audio
duration = 5  # seconds
filename = "output.wav"
fs = 44100  # Sample rate

print(f"Recording for {duration} seconds...")
recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
sd.wait()  # Wait until recording is finished
sf.write(filename, recording, fs)
print(f"Recording saved to {filename}")

Recording for 5 seconds...
Recording saved to output.wav


In [7]:
!pip install --upgrade tqdm

Defaulting to user installation because normal site-packages is not writeable
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.40.0
    Uninstalling tqdm-4.40.0:
      Successfully uninstalled tqdm-4.40.0


In [16]:
# Initialize the Whisper model
model = faster_whisper.WhisperModel("small", compute_type="int8")

In [ ]:
import faster_whisper
import sounddevice as sd
import numpy as np

# Define the audio stream callback
def audio_callback(indata, frames, time, status):
    if status:
        print(f"Status: {status}")
    # Convert audio data to the required format
    audio_data = indata[:, 0].astype(np.float32) / 32768.0  # Normalize int16 to float32
    # Transcribe 1 second of audio at a time
    segments, _ =(audio_data, beam_size=5, without_timestamps=True)
    for segment in segments:
        print(f"Text: {segment.text}")

# Start the audio stream
chunk_duration = 1  # seconds
chunk_frames = int(chunk_duration * fs)
print(f"Recording and transcribing in {chunk_duration}-second chunks for {duration} seconds...")
with sd.InputStream(callback=audio_callback, channels=1, samplerate=fs, dtype='int16', blocksize=chunk_frames):
    sd.sleep(duration * 1000)  # Keep the stream open for the specified duration
print("Finished.")

Recording and transcribing in 1-second chunks for 5 seconds...
Text:  ვვ ვვ ვვ ვვ ვვ ვვ ვ ზ ი ვ ჵ ოვ ი მ ზე


KeyboardInterrupt: 

In [ ]:
# USing faster whisper do text to speech
import faster_whisper
import sounddevice as sd
import numpy as np
import torch
from faster_whisper import WhisperModel
import soundfile as sf
import time
import os
import tempfile
import shutil
from gtts import gTTS


# Record live audio 
from pyaudio import PyAudio, paInt16
import wave
import os
import time
import sys
import soundfile as sf
import sounddevice as sd



# Record live audio
duration = 5  # seconds
filename = "output.wav"
fs = 44100  # Sample rate

print(f"Recording for {duration} seconds...")
recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
sd.wait()  # Wait until recording is finished
sf.write(filename, recording, fs)
print(f"Recording saved to {filename}")

# Load the model
model = WhisperModel("small", compute_type="int8")
# Faster-whisper is for speech-to-text (not text-to-speech)
# For text-to-speech we need to use a dedicated TTS library
# Let's install and use gTTS (Google Text-to-Speech)


def text_to_speech(text, output_file="tts_output.mp3"):
    """Convert text to speech using Google's Text-to-Speech API"""
    tts = gTTS(text=text, lang='en')
    tts.save(output_file)
    print(f"Text-to-speech saved to {output_file}")
    
    # Play the generated audio
    data, samplerate = sf.read(output_file)
    sd.play(data, samplerate)
    sd.wait()

# Example usage
sample_text = "This is a demonstration of text to speech conversion using Google's TTS service."
text_to_speech(sample_text)

# You can also convert transcribed text to speech
# For example, after transcribing with faster-whisper:
# text_to_speech(transcription_result)


In [ ]:
import pyttsx3
import platform
import time

def speak_text(text):
    try:
        # Initialize the TTS engine
        engine = pyttsx3.init()
        
        # Adjust properties like rate and volume
        rate = engine.getProperty('rate')
        engine.setProperty('rate', rate)  # Slow down the speech
        
        # Set volume
        engine.setProperty('volume', 1.0)
        
        # Choose a voice
        voices = engine.getProperty('voices')
        if voices:
            engine.setProperty('voice', 'en-us')  # Set to a specific voice (e.g., 'en-us')
        
        # Speak the text
        engine.say(text)
        
        # This is important - keep a reference to the engine until it's done
        engine.runAndWait()
        
        # Add a small delay to ensure processing is complete
        time.sleep(0.5)
        
        # Explicitly stop and dispose of the engine
        engine.stop()
        
        return True
    except Exception as e:
        print(f"Error in text-to-speech: {e}")
        return False

# Use the existing text variable
print(f"Speaking: {text}")
speak_result = speak_text(text)
print(f"Speech completed successfully: {speak_result}")


Speaking: Hello, this is an offline text-to-speech demonstration using pyttsx3.
Voice 0: Afrikaans - ['af']
Voice 1: Amharic - ['am']
Voice 2: Aragonese - ['an']
Voice 3: Arabic - ['ar']
Voice 4: Assamese - ['as']
Voice 5: Azerbaijani - ['az']
Voice 6: Bashkir - ['ba']
Voice 7: Bulgarian - ['bg']
Voice 8: Bengali - ['bn']
Voice 9: Bishnupriya Manipuri - ['bpy']
Voice 10: Bosnian - ['bs']
Voice 11: Catalan - ['ca']
Voice 12: Chinese (Mandarin) - ['cmn']
Voice 13: Czech - ['cs']
Voice 14: Welsh - ['cy']
Voice 15: Danish - ['da']
Voice 16: German - ['de']
Voice 17: Greek - ['el']
Voice 18: English (Caribbean) - ['en-029']
Voice 19: English (Great Britain) - ['en-gb']
Voice 20: English (Scotland) - ['en-gb-scotland']
Voice 21: English (Lancaster) - ['en-gb-x-gbclan']
Voice 22: English (West Midlands) - ['en-gb-x-gbcwmd']
Voice 23: English (Received Pronunciation) - ['en-gb-x-rp']
Voice 24: English (America) - ['en-us']
Voice 25: Esperanto - ['eo']
Voice 26: Spanish (Spain) - ['es']
Voice 2